In [1]:
from IPython.display import display
from IPython.core.display import HTML 
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)
 

import pandas as pd
import numpy as np
import sqlite3
import unicodecsv as csv
import os
from os import listdir
from os.path import isdir, isfile, join


Toggle code

In [2]:
con = sqlite3.connect('ASX.db')
cur = con.cursor()

In [3]:
cur.execute('''CREATE TABLE IF NOT EXISTS stocks
             (symbol TEXT, date INTEGER, open REAL, high REAL, low REAL, close REAL, volume INTEGER)''')


In [4]:
def write_many_to_db(cur, con, tablename, rows):

        if rows:
            numfields = len(rows[0])

            for attempt in range(0,3):

                try:
                    parameterisedquery = "INSERT INTO " + tablename + " VALUES(" + ','.join(['?']*numfields) + ")"
                    cur.executemany(parameterisedquery, rows)
                    con.commit()
                    #print len(rows), 'rows added to ', tablename
                    break

                except Exception as err:
                    print rows
                    raise err

            else:
                raise Exception()

In [5]:
file_path = r'C:\Users\Aaron\Documents\Share Price Data\Raw'
folders = [f for f in listdir(file_path) if isdir(join(file_path, f))]

In [6]:
print folders

['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007 - 2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']


In [7]:
filelist = []
for folder in folders:
    files = [f for f in listdir(join(file_path, folder)) if isfile(join(file_path, folder, f))]
    for filename in files:
        #filename = filename.lower()
        if 'txt' in filename.lower():
            filelist.append(join(file_path, folder, filename))
    print len(filelist)

246
496
751
1004
1252
1505
1758
2013
2266
2518
3023
3276
3529
3781
4034
4287
4540
4794
5034


In [8]:
rowcount = 0
for pricefile in filelist:
    with open(pricefile, 'rb') as data:

        csvreader = csv.reader(data, delimiter=',', quotechar='"')

        rows = [row for row in csvreader]

        rowsfordb = []
        for row in rows:
            if len(row[0]) <= 3:
                if float(row[3]) < max(float(row[2]), float(row[4]), float(row[5])):
                    row[3] = max(float(row[2]), float(row[4]), float(row[5]))
                if float(row[4]) < min(float(row[2]), float(row[3]), float(row[5])):
                    row[4] = min(float(row[2]), float(row[3]), float(row[5]))
                rowsfordb.append(row)
        write_many_to_db(cur, con, 'stocks', rowsfordb)
        rowcount += len(rowsfordb)
        
print rowcount, 'rows added to database'

6039511 rows added to database


In [9]:
stock = raw_input('Input a stock code: ')

Input a stock code: BHP


In [10]:
query = '''SELECT *
FROM stocks
WHERE Symbol = '%s'
AND date BETWEEN '20100101' AND '20161130'
''' % stock

In [11]:
df = pd.read_sql_query(query, con)

In [12]:
df.head()

,symbol,date,open,high,low,close,volume
0,BHP,20100104,42.75,43.26,42.75,43.08,3691070
1,BHP,20100105,43.50,43.64,43.28,43.28,9104798
2,BHP,20100106,43.51,43.85,43.51,43.82,6349781
3,BHP,20100107,43.91,44.14,43.77,43.77,6641602
4,BHP,20100108,43.69,43.87,43.62,43.62,4512798
